# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.sparse as sp

from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, jaccard_score

import networkx as nx
# import dgl

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.transforms import NormalizeFeatures,to_undirected
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
import torch_geometric.utils as utils

In [2]:
# Setting up GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Graph Creation

In [3]:
node = pd.read_csv('node_information.csv', header=None)
train = pd.read_csv('train.txt', header=None, names=['Source','Target','Edge'], delim_whitespace=True)
test = pd.read_csv('test.txt', header=None, names=['Source','Target'], delim_whitespace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'node_information.csv'

In [4]:
train 

,Source,Target,Edge
0,939,3809,1
1,2442,5784,1
2,179,3809,1
3,857,2280,1
4,1358,5722,1
...,...,...,...
10491,81,6464,0
10492,4,6433,0
10493,3341,6717,1
10494,1562,4656,0


In [5]:
node

,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3592,7584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3593,7589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3594,7593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3595,7594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
node.loc[:, node.columns != node.columns[0]] = node.loc[:, node.columns != node.columns[0]].astype(int)
node.head()

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\3572210986.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  node.loc[:, node.columns != node.columns[0]] = node.loc[:, node.columns != node.columns[0]].astype(int)


,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
nodes = [i for i in node[0]]
nodes[:10]

[0, 4, 5, 6, 7, 9, 10, 11, 13, 17]

In [8]:
train.head()

,Source,Target,Edge
0,939,3809,1
1,2442,5784,1
2,179,3809,1
3,857,2280,1
4,1358,5722,1


In [62]:
# Creates the oriented graph
diG=nx.DiGraph()

# add the list of all nodes
diG.add_nodes_from(nodes)

# adds the corresponding links between two nodes in trainset
for i in range(train.shape[0]):
    if train['Edge'][i] == 1:
        diG.add_edge(train['Source'][i], train['Target'][i])

# Create a dictionary containing node features
node_attributes = {}
for _, row in node.iterrows():
    node_attributes[row[0]] = torch.from_numpy(np.array(row[1:])).to(torch.float)  

# Add the attributes to nodes
nx.set_node_attributes(diG, node_attributes, "x")


print("The number of nodes: {}".format(diG.number_of_nodes()))
print("The number of edges: {}".format(diG.number_of_edges()))

The number of nodes: 3597
The number of edges: 5248


In [63]:
# Transform networkx graph to PyG graph
G = utils.from_networkx(diG, group_node_attrs=['x'])
print(G)

Data(edge_index=[2, 5248], x=[3597, 932])


# Train-Val Split

In [112]:
split = T.RandomLinkSplit(
    num_val=0.10,
    num_test=0.2,
    is_undirected=False,
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0,
)
train_data, val_data, test_data = split(G)
print('train_data:', train_data)
print('val_data:', val_data)

train_data: Data(edge_index=[2, 3675], x=[3597, 932], edge_label=[7350], edge_label_index=[2, 7350])
val_data: Data(edge_index=[2, 3675], x=[3597, 932], edge_label=[1048], edge_label_index=[2, 1048])


# GNN

In [113]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=200
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [114]:
model = Net(G.x.shape[1], 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

model = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=100)

Epoch: 010, Train Loss: 0.684, Val AUC: 0.629
Epoch: 020, Train Loss: 0.636, Val AUC: 0.513
Epoch: 030, Train Loss: 0.586, Val AUC: 0.525
Epoch: 040, Train Loss: 0.550, Val AUC: 0.522
Epoch: 050, Train Loss: 0.521, Val AUC: 0.525
Epoch: 060, Train Loss: 0.510, Val AUC: 0.529
Epoch: 070, Train Loss: 0.500, Val AUC: 0.531
Epoch: 080, Train Loss: 0.484, Val AUC: 0.538
Epoch: 090, Train Loss: 0.477, Val AUC: 0.534
Epoch: 100, Train Loss: 0.467, Val AUC: 0.535


In [115]:
@torch.no_grad()
def test_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return out, roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [116]:
test_preds = test_link_predictor(model, test_data)
print(f"Test: {test_auc:.3f}")

Test: 0.908


# Make Predictions

In [151]:
new_diG = nx.DiGraph()
new_diG.add_nodes_from(nodes)

# Create a dictionary containing node features
node_attributes = {}
for _, row in node.iterrows():
    node_attributes[row[0]] = torch.from_numpy(np.array(row[1:])).to(torch.float)  


nx.set_node_attributes(new_diG, node_attributes, "x")


new_G = utils.from_networkx(new_diG, group_node_attrs=['x'])
print(new_G)

Data(edge_index=[2, 0], x=[3597, 932])


In [153]:
# Create a dictionary to map node names to indices
node_index = {node: i for i, node in enumerate(new_diG.nodes())}

link_probs = []
model.eval()

for i in range(test.shape[0]):
    # Convert source and target node names to indices
    source_idx = node_index[test['Source'][i]]
    target_idx = node_index[test['Target'][i]]
    
    source = test.iloc[i]['Source']
    target = test.iloc[i]['Target']
    new_diG.add_edge(source, target)
    new_G = utils.from_networkx(new_diG, group_node_attrs=['x'])
    new_G = new_G.to(device)
    
    # Encode the source and target nodes
    z = model.encode(new_G.x, new_G.edge_index)

    # Decode the link probability between the source and target nodes
    link_prob = model.decode(z, torch.tensor([[source_idx], [target_idx]]).to(device)).view(-1).sigmoid()
    link_probs.append(link_prob.cpu().item())

# Print the link probabilities
print(link_probs)

[0.9949101805686951, 0.9997748732566833, 0.9944939017295837, 0.9890902638435364, 0.9286478757858276, 0.9926304817199707, 0.8968269228935242, 0.9259406924247742, 0.8969401717185974, 0.9945998191833496, 0.9905086755752563, 0.992398202419281, 0.8414888381958008, 0.9907252192497253, 0.972545862197876, 0.9896751046180725, 0.9986361861228943, 0.9977772831916809, 0.9999885559082031, 0.9999990463256836, 0.9983515739440918, 0.8105587363243103, 0.999255359172821, 0.9974290728569031, 0.8635630011558533, 0.9025455117225647, 0.9947233200073242, 0.9953719973564148, 0.8696386814117432, 0.9405235052108765, 0.9866443276405334, 0.9965810179710388, 0.999991774559021, 0.9999369382858276, 0.8217225670814514, 0.9880276918411255, 0.9908113479614258, 0.997499406337738, 0.9981982111930847, 0.9963571429252625, 0.8104762434959412, 0.898521363735199, 0.9779817461967468, 0.9934701919555664, 0.9981766939163208, 0.7715267539024353, 0.999832034111023, 0.9018988609313965, 0.7343870997428894, 0.785319447517395, 0.78783

In [154]:
test_predictions = pd.DataFrame(columns = ['Source', 'Target', 'Edge'], dtype=object)
# Iterate through rows
for i in range(len(link_probs)):
    source_val = test['Source'][i] 
    target_val = test['Target'][i]
    edge_val = 1 if link_probs[i] >= 0.5 else 0
    
    # Add the row to the DataFrame
    test_predictions = test_predictions.append({
        'Source': source_val,
        'Target': target_val,
        'Edge': edge_val
    }, ignore_index=True)

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

...

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\antoi\AppData\Local\Temp\ipykernel_36948\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\ant

In [155]:
test_predictions[test_predictions['Edge']==1]

,Source,Target,Edge
0,3425,4524,1
1,1620,2617,1
2,4832,6317,1
3,4984,7298,1
4,385,5481,1
...,...,...,...
3493,1548,2957,1
3494,717,1756,1
3495,1731,3976,1
3496,426,1120,1


In [156]:
test_predictions

,Source,Target,Edge
0,3425,4524,1
1,1620,2617,1
2,4832,6317,1
3,4984,7298,1
4,385,5481,1
...,...,...,...
3493,1548,2957,1
3494,717,1756,1
3495,1731,3976,1
3496,426,1120,1


In [157]:
# Join the two DataFrames
preds = pd.merge(test, test_predictions, on=['Source', 'Target'])

In [158]:
preds

,Source,Target,Edge
0,3425,4524,1
1,1620,2617,1
2,4832,6317,1
3,4984,7298,1
4,385,5481,1
...,...,...,...
3493,1548,2957,1
3494,717,1756,1
3495,1731,3976,1
3496,426,1120,1


# Submission

In [159]:
submission = pd.DataFrame()
submission['ID'] = list(range(preds.shape[0]))
submission['Predicted'] = list(preds['Edge'])
submission.to_csv('submission.csv', index = False)